In [1]:
import pandas as pd

df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33780977 entries, 2011-04-06 00:00:00 to 2011-04-06 00:00:00
Data columns (total 11 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   Unique Key              int64 
 1   Agency                  object
 2   Agency Name             object
 3   Complaint Type          object
 4   Descriptor              object
 5   Location Type           object
 6   Incident Zip            object
 7   City                    object
 8   Resolution Description  object
 9   Borough                 object
 10  Open Data Channel Type  object
dtypes: int64(1), object(10)
memory usage: 3.0+ GB


In [ ]:
df.isnull().sum()

In [2]:
# Convert to 'category' for columns with limited unique text values
for col in ['Agency', 'Agency Name', 'Complaint Type', 'Location Type', 'City', 'Borough', 'Open Data Channel Type']:
    df[col] = df[col].astype('category')

# Convert 'Incident Zip' to a more appropriate type (if it contains numeric values)
df['Incident Zip'] = pd.to_numeric(df['Incident Zip'], errors='coerce', downcast='integer')

# Optimizing integer columns
df['Unique Key'] = pd.to_numeric(df['Unique Key'], downcast='integer')

# Memory usage before and after optimization
print("Memory usage before optimization:")
print(df.memory_usage(deep=True).sum())
print("\nMemory usage after optimization:")
print(df.memory_usage(deep=True).sum())


Memory usage before optimization:
10517987169

Memory usage after optimization:
10517987169


In [ ]:
start_date = df.index.min()
end_date = df.index.max()

print("Start Date:", start_date)
print("End Date:", end_date)

In [3]:
# Resample to get daily counts
daily_counts = df.resample('D')['Unique Key'].count()

# Filter for 2022
daily_counts_2022 = daily_counts[daily_counts.index.year == 2022]

# Calculate the average
average_daily_complaints_2022 = daily_counts_2022.mean()

print("Average number of daily complaints in 2022:", average_daily_complaints_2022)

Average number of daily complaints in 2022: 8684.320547945206


In [4]:
max_complaints_date = daily_counts.idxmax()
max_complaints = daily_counts.max()

print(f"The maximum number of calls were received on {max_complaints_date.date()} with a total of {max_complaints} calls.")

The maximum number of calls were received on 2020-08-04 with a total of 24415 calls.


In [6]:
# Filtering the data for that specific date
data_on_max_calls_date = df.loc[max_complaints_date.strftime('%Y-%m-%d')]

# Finding the most common complaint type on that date
most_common_complaint = data_on_max_calls_date['Complaint Type'].value_counts().idxmax()

most_common_complaint, max_complaints_date.date()

('Damaged Tree', datetime.date(2020, 8, 4))

In [8]:
# Resample the DataFrame to get monthly counts of unique keys
monthly_counts = df.resample('M')['Unique Key'].count()

# Find the month with the fewest number of calls
quietest_month = monthly_counts.idxmin()

print(f"The quietest month historically is {quietest_month.strftime('%B %Y')}, with the fewest number of calls.")


The quietest month historically is August 2023, with the fewest number of calls.


In [9]:
import statsmodels.api as sm

# Perform ETS decomposition
decomposed = sm.tsa.seasonal_decompose(daily_counts, model='additive')

# Get the seasonal component
seasonal_component = decomposed.seasonal

# Find the value of the seasonal component on 2020-12-25
seasonal_value_on_2020_12_25 = seasonal_component.loc['2020-12-25']

# Print the rounded value
rounded_value = round(seasonal_value_on_2020_12_25)
print(f"The rounded value of the seasonal component on 2020-12-25 is {rounded_value}.")

The rounded value of the seasonal component on 2020-12-25 is 183.


In [10]:
autocorrelation_lag_1 = daily_counts.autocorr(lag=1)

print(f"The autocorrelation of the number of daily calls with a lag of 1 day is {autocorrelation_lag_1}.")

The autocorrelation of the number of daily calls with a lag of 1 day is 0.7517059728398577.


In [11]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt

# Prepare the data in the format required by Prophet
df_prophet = daily_counts.reset_index()
df_prophet.columns = ['ds', 'y']

# Split the data into training and test sets
train = df_prophet.iloc[:-90]
test = df_prophet.iloc[-90:]

# Initialize and fit the Prophet model
model = Prophet()
model.fit(train)

# Create a dataframe for future predictions
future = model.make_future_dataframe(periods=90)

# Make predictions
forecast = model.predict(future)

# Extract predictions corresponding to the test set
predictions = forecast.iloc[-90:]['yhat']

# Calculate RMSE
rmse = sqrt(mean_squared_error(test['y'], predictions))
print(f"RMSE on the test set is {rmse}.")


19:50:45 - cmdstanpy - INFO - Chain [1] start processing
19:50:46 - cmdstanpy - INFO - Chain [1] done processing


RMSE on the test set is 1231.513760758433.
